In [1]:
import pandas as pd
import numpy as np
from kmapper import KeplerMapper
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from kmapper.cover import Cover
from sklearn.cluster import DBSCAN

/home/ace/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
#read in big names, row labled by year-author-subgenre-title
df = pd.read_csv("SuperScience/vep_super_science_v2_ubiq321_ds.csv", index_col=0)

#drop non-numeric columns
columns_to_drop = ['text_key', 'html_name', 'chunk_index', '!UNRECOGNIZED', '!UNTAGGED', '!BLACKLISTED', '<# Word Tokens>', '<# Punctuation Tokens>', '<# Tokens>']
df = df.drop(columns=columns_to_drop)

df.head(10)

,AbstractConcepts,Acknowledge,Anger,Apology,Aside,Attack_Citation,Authoritative_Citation,Autobio,Biographical_Time,Cause,...,SubjectivePercept,SubjectiveTime,Substitution,Support,TimeDate,TimeDuration,TimeShift,Transformation,Uncertainty,Updates
text_name,,,,,,,,,,,,,,,,,,,,,
A00291.headed.txt,2.337789,0.006104,0.018312,0.000000,0.482207,0.0,0.000000,0.000000,0.030519,0.335714,...,0.647012,0.067143,0.018312,0.042727,0.024416,1.037661,0.274675,0.482207,0.402857,0.134286
A00354.headed.txt,3.210962,0.061356,0.184068,0.000000,0.756724,0.0,0.010226,0.030678,0.337458,0.214746,...,1.840679,0.317006,0.030678,0.081808,0.000000,0.490848,0.092034,0.429492,0.705594,0.378362
A00425.headed.txt,4.774546,0.001258,0.037743,0.000000,0.788838,0.0,0.005032,0.002516,0.088068,0.236526,...,0.952393,0.188717,0.016355,0.007549,0.015097,0.929747,0.260430,0.669317,0.763676,0.213880
A00429.headed.txt,5.880564,0.001389,0.002779,0.000463,0.460985,0.0,0.001698,0.007565,0.042918,0.076728,...,1.213905,0.035199,0.009109,0.008028,0.001235,0.164571,0.056041,0.246857,0.360173,0.098187
A00453.headed.txt,5.073393,0.000000,0.213767,0.000000,0.342027,0.0,0.000000,0.000000,0.042753,0.085507,...,0.798062,0.114009,0.000000,0.000000,0.000000,0.498789,0.057004,0.370529,0.399031,0.299273
A00462.headed.txt,2.950820,0.019286,0.096432,0.000000,0.829315,0.0,0.000000,0.231437,0.173578,0.520733,...,1.928640,0.347155,0.000000,0.096432,0.192864,1.639344,0.482160,0.462874,0.752170,0.443587
A00472.headed.txt,3.380282,0.018884,0.166811,0.000000,0.835628,0.0,0.001574,0.062948,0.199858,0.346211,...,1.775120,0.286411,0.004721,0.020458,0.012590,1.325045,0.350932,0.511449,1.180266,0.387127
A00508.headed.txt,5.440545,0.019191,0.069566,0.000000,0.913954,0.0,0.009595,0.009595,0.357426,0.582916,...,1.885480,0.283062,0.019191,0.045578,0.009595,0.810804,0.280663,0.729244,0.940341,0.278264
A00515.headed.txt,2.784539,0.048681,0.000000,0.000000,0.350501,0.0,0.000000,0.000000,0.223931,0.467335,...,1.012560,0.467335,0.009736,0.038945,0.107098,1.655146,1.022296,1.119657,0.759420,0.194723


In [3]:
#scale the data
X = df.select_dtypes(include="number")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) #take X and scale to [0,1] with std dev 1


In [4]:
#mapper takes np arrays, but we should preserve row names 

X_scaled = pd.DataFrame(
    X_scaled,
    index=df.index,
    columns=df.columns
)

#check it
X_scaled.head(3)


,AbstractConcepts,Acknowledge,Anger,Apology,Aside,Attack_Citation,Authoritative_Citation,Autobio,Biographical_Time,Cause,...,SubjectivePercept,SubjectiveTime,Substitution,Support,TimeDate,TimeDuration,TimeShift,Transformation,Uncertainty,Updates
text_name,,,,,,,,,,,,,,,,,,,,,
A00291.headed.txt,-1.417809,-0.405119,-0.786970,-0.226867,-0.459604,-0.276041,-0.516989,-0.561274,-0.996727,0.405030,...,-1.362960,-1.452522,-0.297055,-0.226160,-0.249243,-0.078933,-0.307183,-0.362729,-0.885202,-1.167718
A00354.headed.txt,-0.921178,1.948848,0.816416,-0.226867,0.381520,-0.276041,0.379086,-0.361305,1.051245,-0.290281,...,0.909047,0.711099,0.070946,0.333432,-0.380727,-0.733989,-1.339923,-0.529961,0.132907,0.329480
A00425.headed.txt,-0.031865,-0.611569,-0.599003,-0.226867,0.479919,-0.276041,-0.076009,-0.544872,-0.612748,-0.165094,...,-0.781703,-0.399781,-0.355267,-0.729876,-0.299424,-0.208209,-0.387732,0.230852,0.328238,-0.679478


In [5]:
#pca two componenets 
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print("Variance explained by each principle comp:", pca.explained_variance_ratio_)
print("Cumulative variance:", pca.explained_variance_ratio_.cumsum())

Variance explained by each principle comp: [0.14819593 0.05655883]
Cumulative variance: [0.14819593 0.20475476]


In [6]:
#dimensionality reduction, project to PCA space with two most significant comps
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# X_pca will be used as the lens/ruler 
lens = X_pca

In [7]:
#Init. Kepler Mapper
mapper = KeplerMapper(verbose=1)

KeplerMapper(verbose=1)


In [8]:
#create cover 
cover = Cover(n_cubes=15, perc_overlap=0.1)

In [9]:
# Create Mapper graph
graph = mapper.map(
    lens,
    X_scaled,
    clusterer=DBSCAN(eps=10, min_samples=3),
    cover=cover
)

Mapping on data shaped (1979, 115) using lens shaped (1979, 2)

Creating 225 hypercubes.

Created 171 edges and 112 nodes in 0:00:00.155779.


In [10]:
tooltips = df.index.astype(str).to_numpy()

mapper.visualize(graph, path_html="./SuperScienceNetwork.html",
                 title="Super Science Network", custom_tooltips=tooltips, include_searchbar=True)

Wrote visualization to: ./SuperScienceNetwork.html


'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Super Science Network | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  font-size: 13px;\n  fo

In [12]:
import json
named_nodes = {}
for key, val in graph['nodes'].items():
    named_nodes[key] = [tooltips[i] for i in val]

with open("SuperScienceGraph.json", "w") as file:
    graph['nodes'] = named_nodes
    json.dump(graph, file)